# Vehicle Input .csv File and Phase_info Dictionary

## Vehicle Input .csv File

```{note}
This section is under development.
```

Aviary uses two main files to define the vehicle and the trajectory -- the vehicle .csv definition and a `phase_info` dictionary definition that defines the trajectory parameters.
You can think of the first file as describing the vehicle's physical properties and the second file as describing the trajectory's properties and optimization parameters.

The vehicle .csv file is structured as multiple rows, each containing a specific vehicle parameter's name, value, and units.
A portion of an example vehicle .csv file is shown below:

```
aircraft:avionics:mass_scaler,1.2,unitless
aircraft:crew_and_payload:baggage_mass_per_passenger,45.0,lbm
aircraft:crew_and_payload:mass_per_passenger,180.0,lbm
aircraft:wing:span,117.83,ft
aircraft:wing:strut_bracing_factor,0.0,unitless
aircraft:wing:surface_ctrl_mass_scaler,1.0,unitless
aircraft:wing:sweep,25.0,deg
...
```

Depending on which analysis options you use with Aviary, you might need to define certain parameters within the vehicle .csv file.
The simplest way to get started is to use a provided example vehicle .csv file and modify it as needed for your specific vehicle.

In [ ]:
from aviary.utils.process_input_decks import create_vehicle
from aviary.utils.aviary_values import AviaryValues
vehicle_deck = AviaryValues()
_ , initial_guesses = create_vehicle(vehicle_deck=vehicle_deck)
for key, val in initial_guesses.items():
    print(f"{key}: {val}")

In the example vehicle input .csv files there is a section headed '# Initial Guesses' which contains the above list in the format: "actual_takeoff_mass,0"
The user can also specify these parameters with the prefix 'initial_guesses:'
e.g. 'initial_guesses:actual_takeoff_mass,150000' would set actual_takeoff_mass in the initial_guesses dictionary to 150000.

If mission_method is TWO_DEGREES_OF_FREEDOM or mass_method is GASP then the initial_guessing() method is called and wherever the initial_guesses values are equal to 0, they are updated with calculated estimates based off the problem type (sizing, alternate, fallout) and mass, speed, range, thrust, and payload data specified in the vehicle input .csv file.

The initial guess of `reserves` is used to define the reserve fuel. Initially, its value can be anything larger than or equal to 0. There are two Aviary variables to control the reserve fuel in the model file (`.csv`):
- `Aircraft.Design.RESERVE_FUEL_ADDITIONAL`: the required fuel reserves: directly in lbm,
- `Aircraft.Design.RESERVE_FUEL_FRACTION`: the required fuel reserves: given as a proportion of mission fuel.

If the value of initial guess of `reserves` (also in the model file if any) is 0, the initial guess of reserve fuel comes from the above two Aviary variables.  Otherwise, it is determined by the parameter `reserves`:
- if `reserves > 10`, we assume it is the actual fuel reserves.
- if `0.0 <= reserves <= 10`, we assume it is the fraction of the mission fuel.

The initial guess of `reserves` is always converted to the actual design reserves (instead of reserve factor) and is used to update the initial guesses of `fuel_burn_per_passenger_mile` and `cruise_mass_final`.

In [ ]:
# Testing Cell
from aviary.utils.process_input_decks import initial_guessing
from aviary.api import Aircraft
Aircraft.Design.RESERVE_FUEL_ADDITIONAL;
Aircraft.Design.RESERVE_FUEL_FRACTION;

## Phase Info Dictionary

`phase_info` is a nested dictionary that Aviary uses for users to define their mission phases - how they are built, the design variables, constraints, connections, etc.

We will now discuss the meaning of the keys within the `phase_info` objects.

- If a key starts with `min_` or `max_` or ends with `_lower` or `_upper`, it is a lower or upper bound of a state variable. The following keys are not state variables:
  - `required_available_climb_rate`: the minimum rate of climb required from the aircraft at the top of climb (beginning of cruise) point in the mission. You don't want your available rate-of-climb to be 0 in case you need to gain altitude during cruise.
  - `EAS_limit`: the maximum descending EAS in knots.
  - `throttle_setting`: the prescribed throttle setting. This is only used for `GASP` and `solved` missions.
- If a key ends with `_ref` or `_ref0` (except `duration_ref`, `duration_ref0`, `initial_ref` and `initial_ref0`), it is the unit-reference and zero-reference values of the control variable at the nodes. This option is invalid if opt=False. Note that it is a simple usage of  ref and ref0. We refer to [Dymos](https://openmdao.github.io/dymos/api/phase_api.html?highlight=ref0#add-state) for details.
- Some keys are for phase time only.
  - `duration_ref` and `duration_ref0` are unit-reference and zero reference for phase time duration.
  - `duration_bounds` are the bounds (lower, upper) for the time duration of the phase.
  - `initial_ref` and `initial_ref0` are the unit-reference and zero references for the initial value of time.
  - `time_initial_ref` and `time_initial_ref0` are the unit-reference and zero-reference for the initial value of time.
  - `initial_bounds`: the lower and upper bounds of initial time. For `GASP`, it is `time_initial_bounds`.
- If a key starts with `final_`, it is the final value of a state variable.
- If a key ends with `_constraint_eq`, it is an equality constraint.

- Keys related to altitude:
  - We use `final_altitude` to indicate the final altitude of the phase.
  - Meanwhile, `alt` is a key in acceleration phase parameter for altitude in `GASP` missions and `altitude` is a key in all other phases of all missions.

- Some keys are a boolean flag of True or False:
  - `input_initial`: the flag to indicate whether initial values of of a state (such as: altitude, velocity, mass, etc.) is taken.
  - `add_initial_mass_constraint`: the flag to indicate whether to add initial mass constraint
  - `clean`: the flag to indicate no flaps or gear are included.
  - `connect_initial_mass`: the flag to indicate whether the initial mass is the same as the final mass of previous phase.
  - `fix_initial`: the flag to indicate whether the initial state variables is fixed.
  - `fix_initial_time`: the flag to indicate whether the initial time is fixed.
  - `no_climb`: if True for the descent phase, the aircraft is not allowed to climb during the descent phase.
  - `no_descent`: if True for the climb phase, the aircraft is not allowed to descend during the climb phase.
  - `include_landing`: the flag to indicate whether there is a landing phase.
  - `include_takeoff`: the flag to indicate whether there is a takeoff phase.
  - `optimize_mass`: if True, the gross takeoff mass of the aircraft is a design variable.
  - `target_mach`: the flag to indicate whether to target mach number.
- `initial_guesses`: initial guesses of state variables.
- `COLLOCATION` related keys:
  - `num_segments`: the number of segments in transcription creation in Dymos. The minimum value is 1. This is needed if 'AnalysisScheme' is `COLLOCATION`.
  - `order`: the order of polynomials for interpolation in transcription creation in Dymos. The minimum value is 3. This is needed if 'AnalysisScheme' is `COLLOCATION`.
- Other Aviary keys:
  - `subsystem_options`: The `aerodynamics` key allows two methods: `computed` and `solved_alpha`. In case of `solved_alpha`, it requires an additional key `aero_data_file`.
  - `external_subsystems`: a list of external subsystems.
- other keys that are self-explanatory:
  - `clean`: a flag for low speed aero (which includes high-lift devices) or cruise aero (clean, because it does not include high-lift devices).
  - `EAS_target`: the target equivalent airspeed.
  - `initial_mach`: initial Mach number.
  - `linear_solver`:  provide an instance of a [LinearSolver](https://openmdao.org/newdocs/versions/latest/features/core_features/controlling_solver_behavior/set_solvers.html) to the phase.
  - `mach_cruise`: the cruise mach number.
  - `mass_f_cruise`: final cruise mass (kg). It is used as `ref` and `defect_ref` in cruise phase.
  - `nonlinear_solver`: provide an instance of a [NonlinearSolver](https://openmdao.org/newdocs/versions/latest/features/core_features/controlling_solver_behavior/set_solvers.html) to the phase.
  - `ode_class`: default to `MissionODE`.
  - `range_f_cruise`: final cruise range (m). It is used as `ref` and `defect_ref` in cruise phase.
  - `solve_segments`: False, 'forward', 'backward'. This is a Radau option.
  - `polynomial_control_order`: default to `None`.
  - `use_actual_takeoff_mass`: default to `False`.
  - `fix_duration`: default to `False`.
  - `solve_for_distance`: if True, use a nonlinear solver to converge the `distance` state variable to the desired value. Otherwise use the optimizer to converge the `distance` state.
  - `optimize_mach`: if True, the Mach number is a design variable.
  - `optimize_altitude`: if True, the altitude is a design variable.
  - `constraints`: a dictionary of user-defined constraints. The keys are the names of the constraints and the values are the keyword arguments expected by Dymos.

```{note}
Not all the keys apply to all phases. The users should select the right keys for each phase of interest. The required keys for each phase are defined in [check_phase_info](https://github.com/OpenMDAO/Aviary/blob/main/aviary/interface/utils.py) function. Currently, this function does the check only for `FLOPS` and `GASP` missions.
```

## Using custom phase builders

For the `height_energy`, you can use a user-defined phase builder.
The user-defined phase builder must inherit from `PhaseBuilderBase` and provide the `from_phase_info` and the `build_phase` methods.
The `from_phase_info` method is used to convert the `phase_info` dictionary into the inputs needed for the phase builder object.
The `build_phase` method is used to actually build and output the `Phase` object.

For examples of how to create a custom phase builder, see the `energy_phase.py` file.

```{note}
Using custom phase builders is a particularly advanced feature and is not recommended for most users.
```

In [ ]:
# Testing Cell
from aviary.mission.phase_builder_base import PhaseBuilderBase
PhaseBuilderBase.from_phase_info;
PhaseBuilderBase.build_phase;